# Trained the pretrained weight

In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K
# os.environ["CUDA_VISIBLE_DEVICES"] = '3'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, Lambda, Dropout
from keras.layers import LeakyReLU, Multiply, add, dot, SeparableConv2D, DepthwiseConv2D, Reshape, Add
from keras.regularizers import l2
from keras.models import Model, load_model, model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from keras import regularizers
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import glob
from PIL import Image
from tqdm import tqdm, trange
import random

Using TensorFlow backend.


In [2]:
nb_classes = 1  # number of classes
img_width, img_height = 64, 64  # change based on the shape/structure of your images
batch_size = 32  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 50  # number of iteration the algorithm gets trained.
learn_rate = 1e-5  # sgd learning rate

# Dir

In [3]:
train_dir = '/home/www/fake_detection/task2/128x128/train'
validation_dir = '/home/www/fake_detection/task2/128x128/validation'
test50_dir = '/home/www/fake_detection/task2/128x128/test_50'

# ShallowNet

In [4]:
# ShallowNet V3
img_input = Input(shape=(img_height, img_width, 3))

# block 1
x = Conv2D(32, (5, 5), padding='valid', kernel_regularizer=regularizers.l2(0.0001))(img_input)
x = Activation('relu')(x)
x = Dropout(0.25)(x)
x = Conv2D(32, (1, 1), kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
x = Dropout(0.25)(x)
x = Conv2D(32, (1, 1), kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

# block 2
x = Conv2D(64, (3, 3), padding='valid', kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
x = Dropout(0.25)(x)
x = Conv2D(64, (1, 1), kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
x = Dropout(0.25)(x)
x = Conv2D(64, (1, 1), kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

# block 3
x = Conv2D(128, (3, 3), padding='valid', kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
x = Dropout(0.25)(x)
x = Conv2D(32, (1, 1), kernel_regularizer=regularizers.l2(0.0001))(x)
x = Activation('relu')(x)
x = Dropout(0.25)(x)

# block 4
x = Flatten()(x)
x_de = Dense(1024, kernel_regularizer=regularizers.l2(0.0001))(x)
x_rl = Activation('relu')(x_de)
x_bn = BatchNormalization()(x_rl)
x_dp = Dropout(0.25)(x_bn)

x_sig = Dense(2, activation=None)(x_dp)
out = Activation('sigmoid')(x_sig)

model = Model(img_input, out)
print("Loaded Model from disk")
print(model.summary())


Loaded Model from disk
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
activation_1 (Activation)    (None, 60, 60, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 60, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 32)        1056      
_________________________________________________________________
activation_2 (Activation)    (None, 60, 60, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None,

In [5]:
model = load_model("/home/www/fake_detection/model/pretrained/SNv3_3.h5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



/home/www/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [6]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(len(model.trainable_weights))

22


# Data generator

In [7]:
train_datagen = ImageDataGenerator(rotation_range=0.0, 
                                   shear_range=0.0,
                                   zoom_range=0.0,
                                   width_shift_range=0.0,
                                   height_shift_range=0.0,
                                   horizontal_flip=False,
                                   rescale=1./255,)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(validation_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        shuffle=False,
                                                        class_mode='categorical')

test50_generator = test_datagen.flow_from_directory(test50_dir,
                                                  target_size=(img_height, img_width),
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  class_mode='categorical')

Found 128404 images belonging to 2 classes.
Found 32100 images belonging to 2 classes.
Found 37566 images belonging to 2 classes.


In [8]:
# callback_list = [EarlyStopping(monitor='val_accuracy', patience=10),
#                  ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)]
# history = model.fit_generator(train_generator,
#                             steps_per_epoch=200,
#                             epochs=100,
#                             validation_data=validation_generator,
#                             validation_steps=len(validation_generator),
#                             callbacks=callback_list)

In [9]:
# model.save('/home/www/fake_detection/model/celeba_shallownetv3.h5')

In [10]:
# model = load_model('/home/www/fake_detection/model/celeba_shallownetv3.h5')

In [11]:
output = model.predict_generator(test50_generator, steps=len(test50_generator), verbose=1)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test50_generator.class_indices)
print(output)

1174/1174 [==============================] - 23s 20ms/step
{'0_real': 0, '1_fake': 1}
[[0.541 0.001]
 [0.556 0.000]
 [0.420 0.001]
 ...
 [0.167 0.978]
 [0.215 0.974]
 [0.232 0.973]]


In [12]:
output_score50 = []
output_class50 = []
answer_class50 = []
answer_class50_1 =[]

for i in trange(len(test50_generator)):
    output50 = model.predict_on_batch(test50_generator[i][0])
    output_score50.append(output50)
    answer_class50.append(test50_generator[i][1])
    
output_score50 = np.concatenate(output_score50)
answer_class50 = np.concatenate(answer_class50)

output_class50 = np.argmax(output_score50, axis=1)
answer_class50_1 = np.argmax(answer_class50, axis=1)

print(output_class50)
print(answer_class50_1)

100%|██████████| 1174/1174 [00:48<00:00, 24.30it/s]


[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]


In [13]:
cm50 = confusion_matrix(answer_class50_1, output_class50)
report50 = classification_report(answer_class50_1, output_class50)

recall50 = cm50[0][0] / (cm50[0][0] + cm50[0][1])
fallout50 = cm50[1][0] / (cm50[1][0] + cm50[1][1])

fpr50, tpr50, thresholds50 = roc_curve(answer_class50_1, output_score50[:, 1], pos_label=1.)
eer50 = brentq(lambda x : 1. - x - interp1d(fpr50, tpr50)(x), 0., 1.)
thresh50 = interp1d(fpr50, thresholds50)(eer50)

print(report50)
print(cm50)
print("AUROC: %f" %(roc_auc_score(answer_class50_1, output_score50[:, 1])))
print(thresh50)
print('test_acc: ', len(output_class50[np.equal(output_class50, answer_class50_1)]) / len(output_class50))

              precision    recall  f1-score   support

           0       0.99      0.72      0.83     18788
           1       0.78      1.00      0.87     18778

    accuracy                           0.86     37566
   macro avg       0.89      0.86      0.85     37566
weighted avg       0.89      0.86      0.85     37566

[[13497  5291]
 [   69 18709]]
AUROC: 0.929057
0.9453254368544813
test_acc:  0.8573177873609115
